In [ ]:
# Import pacchetti
from bs4 import BeautifulSoup
import requests, lxml
import pandas as pd
import regex as re
import numpy as np
from functools import reduce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Scraping posizione in classifica: wikipedia

In [ ]:
edizione = []
for i in range(2013, 2023):
  i = str(i)
  edizione.append(i)

In [ ]:
# Scraping tabelle wikipedia
# la query è la pagina wikipedia delle varie edizioni di sanremo
data_total = []

for i in range(0, len(edizione)):
  # definizione query, richiesta e formattazione in html
  query = 'https://it.wikipedia.org/wiki/Festival_di_Sanremo_' + edizione[i]
  html = requests.get(query).text
  soup = BeautifulSoup(html, 'html.parser')

  # le tabelle di interesse sono in varie posizioni e dipendono dall'anno
  # si selezionano tabelle con indici diversi, trovati osservando inspect html
  if (edizione[i]=='2013' or edizione[i]=='2014' or edizione[i]=='2018' or edizione[i]=='2020' or edizione[i]=='2021'):
    values = soup.find_all('table')[4]
  elif (edizione[i]=='2015' or edizione[i]=='2016' or edizione[i]=='2017'):
    values = soup.find_all('table')[5]
  elif (edizione[i]=='2019' or edizione[i]=='2022'):
    values = soup.find_all('table')[3]
  
  # creazione di dataframe edizione con colonne necessarie
  df = np.array(pd.read_html(str(values)))
  data = pd.DataFrame(df[0], columns =['Posizione', 'Interprete', 'titolo', 'autori'])

  #aggiunta colonna edizione per matching successivo
  anno_edizione = int(edizione[i])
  data['edizione'] = anno_edizione

  #si appendono tutte le tabelle in un array
  data_total.append(data)

In [ ]:
# separazione dei dataset per edizione, per poterli controllare singolarmente
data_2013, data_2014, data_2015, data_2016, data_2017, data_2018, data_2019, data_2020, data_2021, data_2022 = data_total

# Scraping autori: recensiamo musica

## 2013

In [ ]:
# query al blog recensiamomusica.com
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2013/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# si seleziona la parte di html necessaria, con la classifica e gli autori
classifica_campioni = str(soup.find_all('p')[7])
classifica_campioni

'<p><strong>1°</strong> Marco Mengoni – <strong>L’essenziale</strong><br/>\n<em>(Roberto Casalino, Francesco De Benedittis, Marco Mengoni)</em><br/>\n<strong>2°</strong> Elio e le Storie Tese – <strong>La canzone mononota</strong><br/>\n<i>(Sergio Conforti, Davide Civaschi, Nicola\xa0Fasani, Elio)</i><br/>\n<strong>3°</strong> Modà – <strong>Se si potesse non morire</strong><br/>\n<em>(Kekko Silvestre)<br/>\n</em><strong>4°</strong> Malika Ayane\xa0– <strong>E se poi</strong><br/>\n<em>(Giuliano Sangiorgi)</em><br/>\n<strong>5°</strong> Raphael Gualazzi – <strong>Sai (ci basta un sogno)</strong><br/>\n<i>(Raphael Gualazzi)</i><br/>\n<strong>6°</strong> Daniele Silvestri – <strong>A bocca chiusa</strong><br/>\n<em>(Daniele Silvestri)</em><br/>\n<strong>7°</strong> Max Gazzè – <strong>Sotto casa</strong><br/>\n<em>(Francesco De Benedittis, Max e Francesco Gazzè)<br/>\n</em><strong>8°</strong> Chiara Galiazzo – <strong>Il futuro che sarà</strong><br/>\n<em>(Francesco Bianconi, Luca Chiara

La prima idea è stata isolare gli autori inserendoli in una lista e pareggiarli con la tabella wikipedia.

Tuttavia non si può creare una lista di tutti gli autori, perchè nonostante le canzoni siano ordinate in ordine di arrivo in classifica, nelle edizioni più vecchie dei brani che non si classificavano alla serata finale. Questi brani NF sono presenti in ordine diverso nella tabella Wikipedia e questa lista in Recensiamomusica. Perciò si ha bisogno di creare un dataframe per ogni edizione con titolo e autori, in modo da poter fare un matching successivamente e avere risultati corretti.

In [ ]:
# si crea una lista di tutti gli autori
# prendendo solo il contenuto tra parentesi tonde
# alcune canzoni hanno parte del titolo tra parentesi; per eliminare quelle istanze si scelgono solo quelle che iniziano con un carattere maiuscolo
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

14


In [ ]:
# lista dei titoli
# si prendono elementi all'interno dei tag <strong>, ma dato che sono presenti anche le posizioni in classifica si prende un elemento si e uno no
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
print(len(lista_titoli_campioni))

14


In [ ]:
# creazione dataset titoli-autori
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,L’essenziale,"Roberto Casalino, Francesco De Benedittis, Mar..."
1,La canzone mononota,"Sergio Conforti, Davide Civaschi, Nicola Fasan..."
2,Se si potesse non morire,Kekko Silvestre
3,E se poi,Giuliano Sangiorgi
4,Sai (ci basta un sogno),Raphael Gualazzi


In [ ]:
# stesso procedimento per i brani che non hanno partecipato alla serata finale
classifica_non_finalisti = str(soup.find_all('p')[8])
classifica_non_finalisti

'<p><strong>Non finalisti</strong><br/>\nMarco Mengoni – <strong>Bellissimo</strong><br/>\n<em>(Gianna Nannini, Pacifico, Davide Tagliapietra)</em><br/>\nElio e le Storie Tese – <strong>Dannati forever</strong><br/>\n<i>(Sergio Conforti, Davide Civaschi, Nicola\xa0Fasani, Elio)</i><br/>\nModà – <strong>Come l’acqua dentro il mare</strong><br/>\n<em>(Kekko Silvestre)<br/>\n</em>Malika Ayane\xa0– <strong>Niente</strong><br/>\n<em>(Giuliano Sangiorgi)</em><br/>\nRaphael Gualazzi – <strong>Senza ritegno</strong><br/>\n<i>(Raphael Gualazzi)</i><br/>\nDaniele Silvestri – <strong>Il bisogno di te (ricatto d’onor)</strong><br/>\n<em>(Daniele Silvestri)</em><br/>\nMax Gazzè – <strong>I tuoi maledettissimi impegni</strong><br/>\n<em>(Matteo Buzzanca, Max e Francesco Gazzè)<br/>\n</em>Chiara Galiazzo – <strong>L’esperienza dell’amore</strong><br/>\n<em>(Federico e Domenico Zampaglione)<br/>\n</em>Annalisa – <strong>Non so ballare</strong><br/>\n<em>(Ermal Meta)<br/>\n</em>Maria Nazionale – <stron

In [ ]:
lista_autori_non_finalisti = re.findall(r'\((.*?)\)', classifica_non_finalisti)
lista_autori_non_finalisti = list(filter(r.match, lista_autori_non_finalisti))
print(len(lista_autori_non_finalisti))

14


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_non_finalisti)
lista_titoli_non_finalisti =x[1:]
len(lista_titoli_non_finalisti)

14

In [ ]:
df_non_finalisti = pd.DataFrame()
df_non_finalisti['titolo']  = lista_titoli_non_finalisti
df_non_finalisti['Autori'] = lista_autori_non_finalisti
df_non_finalisti.head()

,titolo,Autori
0,Bellissimo,"Gianna Nannini, Pacifico, Davide Tagliapietra"
1,Dannati forever,"Sergio Conforti, Davide Civaschi, Nicola Fasan..."
2,Come l’acqua dentro il mare,Kekko Silvestre
3,Niente,Giuliano Sangiorgi
4,Senza ritegno,Raphael Gualazzi


In [ ]:
# dataset totale dell'edizione, con brani finalisti e non
df = pd.concat([df_campioni, df_non_finalisti], ignore_index=True)

In [ ]:
# controllo che i dataset da unire abbiano le stesse dimensioni
print(len(df))
print(len(data_2013))

28
28


In [ ]:
# preprocessing semplice per facilitare match
df['titolo'] = df['titolo'].str.lower()
data_2013['titolo'] = data_2013['titolo'].str.lower()

# alcune regole che si è notato potrebbero ripetersi
df['titolo'] = df['titolo'].str.replace('\’', '\'')
df['titolo'] = df['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
# merge dei dataset di wikipedia e di recensiamomusica
temp_2013 = pd.merge(data_2013, df, on = ['titolo'], how='left')
print(temp_2013.shape)
temp_2013

(28, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Marco Mengoni,l'essenziale,"R. Casalino, M. Mengoni e F. De Benedittis",2013,"Roberto Casalino, Francesco De Benedittis, Mar..."
1,2º,Elio e le Storie Tese,la canzone mononota,"S. Belisari, S. Conforti, D. L. Civaschi e N. ...",2013,"Sergio Conforti, Davide Civaschi, Nicola Fasan..."
2,3º,Modà,se si potesse non morire,F. Silvestre,2013,Kekko Silvestre
3,4º,Malika Ayane,e se poi,G. Sangiorgi,2013,Giuliano Sangiorgi
4,5º,Raphael Gualazzi,sai (ci basta un sogno),R. Gualazzi,2013,Raphael Gualazzi
5,6º,Daniele Silvestri,a bocca chiusa,D. Silvestri,2013,Daniele Silvestri
6,7º,Max Gazzè,sotto casa,"F. Gazzè, M. Gazzè e F. De Benedittis",2013,"Francesco De Benedittis, Max e Francesco Gazzè"
7,8º,Chiara,il futuro che sarà,"F. Bianconi, L. Chiaravalli e L. Gonzalez Alea",2013,"Francesco Bianconi, Luca Chiaravalli, Lisette ..."
8,9º,Annalisa,scintille,D. Faini e A. Galbiati,2013,"Dario Faini, Antonio Galbiati"
9,10º,Maria Nazionale,è colpa mia,G. Servillo e F. Mesolella,2013,"Peppe Servillo, Fausto Mesolella"


In [ ]:
# scelta colonna autori con nomi per intero
temp_2013 = temp_2013.drop('autori', 1)
temp_2013.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Marco Mengoni,l'essenziale,2013,"Roberto Casalino, Francesco De Benedittis, Mar..."
1,2º,Elio e le Storie Tese,la canzone mononota,2013,"Sergio Conforti, Davide Civaschi, Nicola Fasan..."
2,3º,Modà,se si potesse non morire,2013,Kekko Silvestre
3,4º,Malika Ayane,e se poi,2013,Giuliano Sangiorgi
4,5º,Raphael Gualazzi,sai (ci basta un sogno),2013,Raphael Gualazzi


Queste operazioni vengono ripetute per tutte e dieci le edizioni con, se necessario, delle piccole aggiunte o cambiamenti. La maggior parte di queste riguardano la creazione della lista dei titoli, che all'interno dei siti non presentava sempre un formato uniforme.

## 2014

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2014/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[6])
classifica_campioni

'<p><strong>1°</strong> Arisa – <strong>Controvento</strong><br/>\n<em>(Giuseppe Anastasi)</em><br/>\n<strong>2°</strong> Raphael Gualazzi &amp; The Bloody Beetroots – <strong>Liberi o no</strong><br/>\n<i>(Leonardo Boccafichi, Bob Cornelius Rifo, Raphael Gualazzi)</i><br/>\n<strong>3°</strong> Renzo Rubino – <strong>Ora</strong><br/>\n<em>(Andrea Rodini, Renzo Rubino)<br/>\n</em><strong>4°</strong> Francesco Renga – <strong>Vivendo adesso</strong><br/>\n<em>(Elisa)</em><br/>\n<strong>5°</strong> Noemi – <strong>Bagnati dal sole</strong><br/>\n<i>(Caroline Ailin, Richard Frenneaus, Noemi)</i><br/>\n<strong>6°</strong> Perturbazione – <strong>L’unica</strong><br/>\n<i>(Tommaso Cerasuolo, Gigi Giancursi,\xa0Rossano e Cristiano Lo Mele, Alex Barocco, Elena Diana)</i><br/>\n<strong>7°</strong> Cristiano De André – <strong>Il cielo è vuoto</strong><br/>\n<em>(Dario Faini, Diego Mancino, Cristiano De André)<br/>\n</em><strong>8°</strong> Frankie hi-nrg mc – <strong>Pedala</strong><br/>\n<em>

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

14


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
print(len(lista_titoli_campioni))

14


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Controvento,Giuseppe Anastasi
1,Liberi o no,"Leonardo Boccafichi, Bob Cornelius Rifo, Rapha..."
2,Ora,"Andrea Rodini, Renzo Rubino"
3,Vivendo adesso,Elisa
4,Bagnati dal sole,"Caroline Ailin, Richard Frenneaus, Noemi"


In [ ]:
classifica_non_finalisti = str(soup.find_all('p')[7])
classifica_non_finalisti

'<p><strong>Non finalisti</strong><br/>\nArisa – <strong>Lentamente (il primo che passa)</strong><br/>\n<em>(Cristina Donà, Saverio Lanza, Arisa)</em><br/>\nRaphael Gualazzi &amp; The Bloody Beetroots – <strong>Tanto ci sei</strong><br/>\n<i>(Giuliano Sangiorgi, Bob Cornelius Rifo, Raphael Gualazzi)</i><br/>\nRenzo Rubino – <strong>Per sempre e poi basta</strong><br/>\n<em>(Andrea Rodini, Renzo Rubino)<br/>\n</em>Francesco Renga – <strong>A un isolato da te</strong><br/>\n<em>(Roberto Casalino)</em><br/>\nNoemi – <strong>Un uomo e un albero</strong><br/>\n<i>(Dario Faini, Diego Mancino, Noemi)</i><br/>\nPerturbazione –\xa0<strong>L’Italia vista dal bar</strong><br/>\n<i>(Tommaso Cerasuolo, Gigi Giancursi,\xa0Rossano e Cristiano Lo Mele, Alex Barocco, Elena Diana)</i><br/>\nCristiano De André – <strong>Invisibili</strong><br/>\n<em>(Fabio Ferraboschi, Cristiano De André)<br/>\n</em>Frankie hi-nrg mc – <strong>Un uomo è vivo</strong><br/>\n<em>(Leonardo Beccafichi, Carolina Galbignani, F

In [ ]:
lista_autori_non_finalisti = re.findall(r'\((.*?)\)', classifica_non_finalisti)
lista_autori_non_finalisti = list(filter(r.match, lista_autori_non_finalisti))
print(len(lista_autori_non_finalisti))

14


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_non_finalisti)
lista_titoli_non_finalisti =x[1:]
len(lista_titoli_non_finalisti)

14

In [ ]:
df_non_finalisti = pd.DataFrame()
df_non_finalisti['titolo']  = lista_titoli_non_finalisti
df_non_finalisti['Autori'] = lista_autori_non_finalisti
df_non_finalisti.head()

,titolo,Autori
0,Lentamente (il primo che passa),"Cristina Donà, Saverio Lanza, Arisa"
1,Tanto ci sei,"Giuliano Sangiorgi, Bob Cornelius Rifo, Raphae..."
2,Per sempre e poi basta,"Andrea Rodini, Renzo Rubino"
3,A un isolato da te,Roberto Casalino
4,Un uomo e un albero,"Dario Faini, Diego Mancino, Noemi"


In [ ]:
df = pd.concat([df_campioni, df_non_finalisti], ignore_index=True)

In [ ]:
print(len(df))
print(len(data_2014))

28
28


In [ ]:
df['titolo'] = df['titolo'].str.lower()
data_2014['titolo'] = data_2014['titolo'].str.lower()

df['titolo'] = df['titolo'].str.replace('\’', '\'')
df['titolo'] = df['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2014 = pd.merge(data_2014, df, on = ['titolo'], how='left')
print(temp_2014.shape)
temp_2014

(28, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Arisa,controvento,G. Anastasi,2014,Giuseppe Anastasi
1,2º,Raphael Gualazzi & The Bloody Beetroots,liberi o no,"R. Gualazzi, B. Rifo, L. Beccafichi",2014,"Leonardo Boccafichi, Bob Cornelius Rifo, Rapha..."
2,3º,Renzo Rubino,ora,"R. Rubino, A. Rodini",2014,"Andrea Rodini, Renzo Rubino"
3,4º,Francesco Renga,vivendo adesso,E. Toffoli,2014,Elisa
4,5º,Noemi,bagnati dal sole,"Noemi, C. Ailin, R. Frenneaux",2014,"Caroline Ailin, Richard Frenneaus, Noemi"
5,6º,Perturbazione,l'unica,"T. Cerasuolo, G. Giancursi, R. Lo Mele, A. Bar...",2014,"Tommaso Cerasuolo, Gigi Giancursi, Rossano e C..."
6,7º,Cristiano De André,il cielo è vuoto,"D. Mancino, C. De André, D. Faini",2014,"Dario Faini, Diego Mancino, Cristiano De André"
7,8º,Frankie hi-nrg mc,pedala,"F. Di Gesú, C. Galbignani, L. Beccafichi",2014,"Leonardo Beccafichi, Carolina Galbignani, Fran..."
8,9º,Giusy Ferreri,ti porto a cena con me,"R. Casalino, D. Faini",2014,"Roberto Casalino, Dario Faini"
9,10º,Francesco Sarcina,nel tuo sorriso,F. Sarcina,2014,Francesco Sarcina


In [ ]:
temp_2014 = temp_2014.drop('autori', 1)
temp_2014.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Arisa,controvento,2014,Giuseppe Anastasi
1,2º,Raphael Gualazzi & The Bloody Beetroots,liberi o no,2014,"Leonardo Boccafichi, Bob Cornelius Rifo, Rapha..."
2,3º,Renzo Rubino,ora,2014,"Andrea Rodini, Renzo Rubino"
3,4º,Francesco Renga,vivendo adesso,2014,Elisa
4,5º,Noemi,bagnati dal sole,2014,"Caroline Ailin, Richard Frenneaus, Noemi"


## 2015

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2015/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[6])
classifica_campioni

'<p><strong>1°</strong> Il Volo – <strong>Grande amore</strong><br/>\n<em>(Francesco Boccia, Ciro Esposito)</em><br/>\n<strong>2°</strong> Nek – <strong>Fatti avanti amore</strong><br/>\n<i>(Luca Chiaravalli, Andrea Bonomo, Gianluigi Fazio, Nek)</i><br/>\n<strong>3°</strong> Malika Ayane – <strong>Adesso e qui (nostalgico presente)</strong><br/>\n<em>(Pacifico, Alessandra Flora, Giovanni Caccamo, Malika Ayane)<br/>\n</em><strong>4°</strong> Annalisa – <strong>Una finestra tra le stelle</strong><br/>\n<em>(Kekko Silvestre)</em><br/>\n<strong>5°</strong> Chiara Galiazzo – <strong>Straordinario</strong><br/>\n<i>(Ermal Meta, Giovanni Pollex)</i><br/>\n<strong>6°</strong> Marco Masini – <strong>Che giorno è</strong><br/>\n<em>(Federica Camba, Daniele Coro, Marco Masini)</em><br/>\n<strong>7°</strong> Dear Jack – <strong>Il mondo esplode tranne noi</strong><br/>\n<em>(Piero Romitelli, Davide Simonetta)<br/>\n</em><strong>8°</strong> Gianluca Grignani – <strong>Sogni infranti</strong><br/>\n

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

16


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
print(len(lista_titoli_campioni))

16


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Grande amore,"Francesco Boccia, Ciro Esposito"
1,Fatti avanti amore,"Luca Chiaravalli, Andrea Bonomo, Gianluigi Faz..."
2,Adesso e qui (nostalgico presente),"Pacifico, Alessandra Flora, Giovanni Caccamo, ..."
3,Una finestra tra le stelle,Kekko Silvestre
4,Straordinario,"Ermal Meta, Giovanni Pollex"


In [ ]:
classifica_non_finalisti = str(soup.find_all('p')[7])
classifica_non_finalisti

'<p><strong>Non finalisti<br/>\n</strong>Raf – <strong>Come una favola</strong><br/>\n<em>(Saverio Grandi, Emiliano Cecere, Raf)<br/>\n</em>Biggio e Mandelli – <strong>Vita d’inferno</strong><br/>\n<i>(Martino Ferro, Francesco\xa0Mandelli, Fabrizio Biggio)</i><br/>\nAnna Tatangelo – <strong>Libera<br/>\n</strong><em>(Kekko Silvestre, Enrico Palmosi)<br/>\n</em>Lara Fabian – <strong>Voce</strong><br/>\n<em>(Fio Zanotti, Cristiano Cremonini, Lara Fabian)</em></p>'

In [ ]:
lista_autori_non_finalisti = re.findall(r'\((.*?)\)', classifica_non_finalisti)
lista_autori_non_finalisti = list(filter(r.match, lista_autori_non_finalisti))
print(len(lista_autori_non_finalisti))

4


In [ ]:
lista_titoli_non_finalisti = re.findall(r'<strong>(.*?)<\/strong>', classifica_non_finalisti)
lista_titoli_non_finalisti.insert(2, "Libera")
len(lista_titoli_non_finalisti)

4

In [ ]:
df_non_finalisti = pd.DataFrame()
df_non_finalisti['titolo']  = lista_titoli_non_finalisti
df_non_finalisti['Autori'] = lista_autori_non_finalisti
df_non_finalisti.head()

,titolo,Autori
0,Come una favola,"Saverio Grandi, Emiliano Cecere, Raf"
1,Vita d’inferno,"Martino Ferro, Francesco Mandelli, Fabrizio Bi..."
2,Libera,"Kekko Silvestre, Enrico Palmosi"
3,Voce,"Fio Zanotti, Cristiano Cremonini, Lara Fabian"


In [ ]:
df = pd.concat([df_campioni, df_non_finalisti], ignore_index=True)

In [ ]:
print(len(df))
print(len(data_2015))

20
20


In [ ]:
df['titolo'] = df['titolo'].str.lower()
data_2015['titolo'] = data_2015['titolo'].str.lower()

df['titolo'] = df['titolo'].str.replace('\’', '\'')
df['titolo'] = df['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2015 = pd.merge(data_2015, df, on = ['titolo'], how='left')
print(temp_2015.shape)
temp_2015

(20, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Il Volo,grande amore,F. Boccia e C. Esposito,2015,"Francesco Boccia, Ciro Esposito"
1,2º,Nek,fatti avanti amore,"Nek, L. Chiaravalli, A. Bonomo e G. Fazio",2015,"Luca Chiaravalli, Andrea Bonomo, Gianluigi Faz..."
2,3º,Malika Ayane,adesso e qui (nostalgico presente),"M. Ayane, Pacifico, G. Caccamo e A. Flora",2015,"Pacifico, Alessandra Flora, Giovanni Caccamo, ..."
3,4º,Annalisa,una finestra tra le stelle,F. Silvestre,2015,Kekko Silvestre
4,5º,Chiara,straordinario,E. Meta e G. Pollex,2015,"Ermal Meta, Giovanni Pollex"
5,6º,Marco Masini,che giorno è,"F. Camba, D. Coro e M. Masini",2015,"Federica Camba, Daniele Coro, Marco Masini"
6,7º,Dear Jack,il mondo esplode tranne noi,P. Romitelli e D. Simonetta,2015,"Piero Romitelli, Davide Simonetta"
7,8º,Gianluca Grignani,sogni infranti,G. Grignani,2015,Gianluca Grignani
8,9º,Nina Zilli,sola,N. Zilli,2015,Nina Zilli
9,10º,Lorenzo Fragola,siamo uguali,"L. Fragola, Fedez e F. Cogliati",2015,"Fedez, Fausto Cogliati, Lorenzo Fragola"


In [ ]:
temp_2015 = temp_2015.drop('autori', 1)
temp_2015.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Il Volo,grande amore,2015,"Francesco Boccia, Ciro Esposito"
1,2º,Nek,fatti avanti amore,2015,"Luca Chiaravalli, Andrea Bonomo, Gianluigi Faz..."
2,3º,Malika Ayane,adesso e qui (nostalgico presente),2015,"Pacifico, Alessandra Flora, Giovanni Caccamo, ..."
3,4º,Annalisa,una finestra tra le stelle,2015,Kekko Silvestre
4,5º,Chiara,straordinario,2015,"Ermal Meta, Giovanni Pollex"


## 2016

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2016/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[6])
classifica_campioni

'<p><strong>1°</strong> Stadio – <strong>Un giorno mi dirai</strong><br/>\n<em>(Gaetano Curreri, Saverio Grandi, Luca Chiaravalli)</em><br/>\n<strong>2°</strong> Francesca Michielin – <strong>Nessun grado di separazione</strong><br/>\n<i>(Federica Abbate, Cheope, Fabio Gargiulo, Francesca Michielin)</i><br/>\n<strong>3°</strong> Giovanni Caccamo e Deborah Iurato – <strong>Via da qui</strong><br/>\n<em>(Giuliano Sangiorgi)<br/>\n</em><strong>4°</strong> Enrico Ruggeri – <strong>Il primo amore non si scorda mai</strong><br/>\n<em>(Enrico Ruggeri)</em><br/>\n<strong>5°</strong> Lorenzo Fragola – <strong>Infinite volte</strong><br/>\n<i>(Rory Di Benedetto, Rosario Canale, Fabio Ferraguzzo, Antonio Filippelli, Lorenzo Fragola)</i><br/>\n<strong>6°</strong> Patty Pravo – <strong>Cieli immensi</strong><br/>\n<em>(Fortunato Zampaglione)</em><br/>\n<strong>7°</strong> Clementino – <strong>Quando sono lontano</strong><br/>\n<em>(Vincent Stein, Konstantin Scherer, Clementino)<br/>\n</em><strong>8

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

16


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
print(len(lista_titoli_campioni))

16


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Un giorno mi dirai,"Gaetano Curreri, Saverio Grandi, Luca Chiaravalli"
1,Nessun grado di separazione,"Federica Abbate, Cheope, Fabio Gargiulo, Franc..."
2,Via da qui,Giuliano Sangiorgi
3,Il primo amore non si scorda mai,Enrico Ruggeri
4,Infinite volte,"Rory Di Benedetto, Rosario Canale, Fabio Ferra..."


In [ ]:
classifica_non_finalisti = str(soup.find_all('p')[7])
classifica_non_finalisti

'<p><strong>Non finalisti<br/>\n</strong>Zero Assoluto – <strong>Di me e di te</strong><br/>\n<em>(Thomas De Gasperi, Matteo Maffucci, Antonio Filippelli, Luca Vicini)<br/>\n</em>Bluvertigo – <strong>Semplicemente</strong><br/>\n<em>(Morgan)<br/>\n</em>Dear Jack – <strong>Mezzo respiro<br/>\n</strong><em>(Leiner Riflessi, Stefano Paviani, Claudio Corradini, Roberto Balbo)<br/>\n</em>Neffa – <strong>Sogni e nostalgia</strong><br/>\n<em>(Neffa)</em></p>'

In [ ]:
lista_autori_non_finalisti = re.findall(r'\((.*?)\)', classifica_non_finalisti)
lista_autori_non_finalisti = list(filter(r.match, lista_autori_non_finalisti))
print(len(lista_autori_non_finalisti))

4


In [ ]:
lista_titoli_non_finalisti = re.findall(r'<strong>(.*?)<\/strong>', classifica_non_finalisti)
lista_titoli_non_finalisti.insert(2, "Mezzo respiro")
len(lista_titoli_non_finalisti)

4

In [ ]:
df_non_finalisti = pd.DataFrame()
df_non_finalisti['titolo']  = lista_titoli_non_finalisti
df_non_finalisti['Autori'] = lista_autori_non_finalisti
df_non_finalisti.head()

,titolo,Autori
0,Di me e di te,"Thomas De Gasperi, Matteo Maffucci, Antonio Fi..."
1,Semplicemente,Morgan
2,Mezzo respiro,"Leiner Riflessi, Stefano Paviani, Claudio Corr..."
3,Sogni e nostalgia,Neffa


In [ ]:
df = pd.concat([df_campioni, df_non_finalisti], ignore_index=True)

In [ ]:
print(len(df))
print(len(data_2016))

20
20


In [ ]:
df['titolo'] = df['titolo'].str.lower()
data_2016['titolo'] = data_2016['titolo'].str.lower()

df['titolo'] = df['titolo'].str.replace('\’', '\'')
df['titolo'] = df['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2016 = pd.merge(data_2016, df, on = ['titolo'], how='left')
print(temp_2016.shape)
temp_2016

(20, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Stadio,un giorno mi dirai,"S. Grandi, G. Curreri e L. Chiaravalli",2016,"Gaetano Curreri, Saverio Grandi, Luca Chiaravalli"
1,2º,Francesca Michielin,nessun grado di separazione,"F. Michielin, F. Abbate, Cheope e F. Gargiulo",2016,"Federica Abbate, Cheope, Fabio Gargiulo, Franc..."
2,3º,Giovanni Caccamo e Deborah Iurato,via da qui,G. Sangiorgi,2016,Giuliano Sangiorgi
3,4º,Enrico Ruggeri,il primo amore non si scorda mai,E. Ruggeri,2016,Enrico Ruggeri
4,5º,Lorenzo Fragola,infinite volte,"L. Fragola, A. Filippelli, R. Di Benedetto, R....",2016,"Rory Di Benedetto, Rosario Canale, Fabio Ferra..."
5,6º,Patty Pravo,cieli immensi,F. Zampaglione,2016,Fortunato Zampaglione
6,7º,Clementino,quando sono lontano,"C. Maccaro, K. Scherer e V. Stein",2016,"Vincent Stein, Konstantin Scherer, Clementino"
7,8º,Noemi,la borsa di una donna,"M. Masini, M. Adami e A. Iammarino",2016,"Marco Masini, Marco Adami, Antonio Iammarino"
8,9º,Rocco Hunt,wake up,"R. Pagliarulo, S. Benussi e V. Catanzaro",2016,"Vincenzo Catanzaro, Simone Benussi, Rocco Hunt"
9,10º,Arisa,guardando il cielo,G. Anastasi,2016,Giuseppe Anastasi


In [ ]:
temp_2016 = temp_2016.drop('autori', 1)
temp_2016.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Stadio,un giorno mi dirai,2016,"Gaetano Curreri, Saverio Grandi, Luca Chiaravalli"
1,2º,Francesca Michielin,nessun grado di separazione,2016,"Federica Abbate, Cheope, Fabio Gargiulo, Franc..."
2,3º,Giovanni Caccamo e Deborah Iurato,via da qui,2016,Giuliano Sangiorgi
3,4º,Enrico Ruggeri,il primo amore non si scorda mai,2016,Enrico Ruggeri
4,5º,Lorenzo Fragola,infinite volte,2016,"Rory Di Benedetto, Rosario Canale, Fabio Ferra..."


## 2017

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2017/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[6])
classifica_campioni

'<p><strong>1°</strong> Francesco Gabbani – <strong>Occidentali’s karma</strong><br/>\n<em>(Fabio Ilaqua, Luca Chiaravalli, Filippo e Francesco Gabbani)</em><br/>\n<strong>2°</strong> Fiorella Mannoia – <strong>Che sia benedetta</strong><br/>\n<i>(Amara, Salvatore Mineo)</i><br/>\n<strong>3°</strong> Ermal Meta – <strong>Vietato Morire</strong><br/>\n<em>(Ermal Meta)<br/>\n</em><strong>4°</strong> Michele Bravi – <strong>Il diario degli errori</strong><br/>\n<em>(Federica Abbate, Cheope, Giuseppe Anastasi)</em><br/>\n<strong>5°</strong> Paola Turci – <strong>Fatti bella per te</strong><br/>\n<i>(Giulia Anania, Davide Simonetta, Luca Chiaravalli, Paola Turci)</i><br/>\n<strong>6°</strong> Sergio Sylvestre – <strong>Con te</strong><br/>\n<em>(Giorgia, Stefano Maiuolo, Sergio Sylvestre)</em><br/>\n<strong>7°</strong> Fabrizio Moro – <strong>Portami via</strong><br/>\n<em>(Roberto Cardelli, Fabrizio Moro)<br/>\n</em><strong>8°</strong> Elodie – <strong>Tutta colpa mia</strong><br/>\n<em>(E

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

16


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
print(len(lista_titoli_campioni))

16


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Occidentali’s karma,"Fabio Ilaqua, Luca Chiaravalli, Filippo e Fran..."
1,Che sia benedetta,"Amara, Salvatore Mineo"
2,Vietato Morire,Ermal Meta
3,Il diario degli errori,"Federica Abbate, Cheope, Giuseppe Anastasi"
4,Fatti bella per te,"Giulia Anania, Davide Simonetta, Luca Chiarava..."


In [ ]:
classifica_non_finalisti = str(soup.find_all('p')[7])
classifica_non_finalisti

'<p><strong>Non finalisti<br/>\n</strong>Al Bano Carrisi\xa0– <strong>Di rose e di spine</strong><br/>\n<em>(Maurizio Fabrizio, Al Bano Carrisi)<br/>\n</em>Gigi D’Alessio – <strong>La prima stella</strong><br/>\n<em>(Gigi D’Alessio)<br/>\n</em>Ron – <strong>L’ottava meraviglia<br/>\n</strong><em>(Mattia Del Forno, Francesco Caprara, Emiliano Mangia, Ron)<br/>\n</em>Giusy Ferreri – <strong>Fa\xa0talmente male</strong><br/>\n<em>(Roberto Casalino, Paolo Catalano, Takagi e Ketra)<br/>\nNesli e Alice Paba – <strong>Di retta a te<br/>\n</strong>(Brando, Nesli)<br/>\nRaige e Giulia Luzi – <strong>Togliamoci la voglia</strong><br/>\n(Luca Chiaravalli, Antonio Iammarino, Zibba, Raige)<br/>\n</em></p>'

In [ ]:
lista_autori_non_finalisti = re.findall(r'\((.*?)\)', classifica_non_finalisti)
lista_autori_non_finalisti = list(filter(r.match, lista_autori_non_finalisti))
print(len(lista_autori_non_finalisti))

6


In [ ]:
lista_titoli_non_finalisti = re.findall(r'<strong>(.*?)<\/strong>', classifica_non_finalisti)
lista_titoli_non_finalisti.insert(2, "L’ottava meraviglia")
lista_titoli_non_finalisti.insert(4, "Do retta a te")
len(lista_titoli_non_finalisti)

6

In [ ]:
df_non_finalisti = pd.DataFrame()
df_non_finalisti['titolo']  = lista_titoli_non_finalisti
df_non_finalisti['Autori'] = lista_autori_non_finalisti
df_non_finalisti.head()

,titolo,Autori
0,Di rose e di spine,"Maurizio Fabrizio, Al Bano Carrisi"
1,La prima stella,Gigi D’Alessio
2,L’ottava meraviglia,"Mattia Del Forno, Francesco Caprara, Emiliano ..."
3,Fa talmente male,"Roberto Casalino, Paolo Catalano, Takagi e Ketra"
4,Do retta a te,"Brando, Nesli"


In [ ]:
df = pd.concat([df_campioni, df_non_finalisti], ignore_index=True)

In [ ]:
print(len(df))
print(len(data_2017))

22
22


In [ ]:
df['titolo'] = df['titolo'].str.lower()
data_2017['titolo'] = data_2017['titolo'].str.lower()

df['titolo'] = df['titolo'].str.replace('\’', '\'')
df['titolo'] = df['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2017 = pd.merge(data_2017, df, on = ['titolo'], how='left')
print(temp_2017.shape)
temp_2017

(22, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Francesco Gabbani,occidentali's karma,"F. Ilacqua, F. Gabbani, L. Chiaravalli e F. Ga...",2017,"Fabio Ilaqua, Luca Chiaravalli, Filippo e Fran..."
1,2º,Fiorella Mannoia,che sia benedetta,Amara e S. Mineo,2017,"Amara, Salvatore Mineo"
2,3º,Ermal Meta,vietato morire,E. Meta,2017,Ermal Meta
3,4º,Michele Bravi,il diario degli errori,"Cheope, F. Abbate e G. Anastasi",2017,"Federica Abbate, Cheope, Giuseppe Anastasi"
4,5º,Paola Turci,fatti bella per te,"G. Anania, L. Chiaravalli, D. Simonetta e P. T...",2017,"Giulia Anania, Davide Simonetta, Luca Chiarava..."
5,6º,Sergio Sylvestre,con te,"G. Todrani, S. Sylvestre e S. Maiuolo",2017,"Giorgia, Stefano Maiuolo, Sergio Sylvestre"
6,7º,Fabrizio Moro,portami via,F. Moro e R. Cardelli,2017,"Roberto Cardelli, Fabrizio Moro"
7,8º,Elodie,tutta colpa mia,"E. Marrone, O. Angiuli, G. Pollex e F. Cianciola",2017,"Emma, Gianni Pollex, Oscar Angiuli, Francesco ..."
8,9º,Bianca Atzei,ora esisti solo tu,F. Silvestre,2017,Bianca Atzei
9,10º,Samuel,vedrai,"Samuel, C. Rigano e R. Onori",2017,"Christian Rigano, Riccardo Onori, Samuel Umber..."


In [ ]:
temp_2017 = temp_2017.drop('autori', 1)
temp_2017.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Francesco Gabbani,occidentali's karma,2017,"Fabio Ilaqua, Luca Chiaravalli, Filippo e Fran..."
1,2º,Fiorella Mannoia,che sia benedetta,2017,"Amara, Salvatore Mineo"
2,3º,Ermal Meta,vietato morire,2017,Ermal Meta
3,4º,Michele Bravi,il diario degli errori,2017,"Federica Abbate, Cheope, Giuseppe Anastasi"
4,5º,Paola Turci,fatti bella per te,2017,"Giulia Anania, Davide Simonetta, Luca Chiarava..."


## 2018

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2018/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[7])
classifica_campioni

'<p><strong>1°</strong> Ermal Meta e Fabrizio Moro – <strong>Non mi avete fatto niente</strong><br/>\n<em>(Andrea Febo, Fabrizio Moro,\xa0Ermal Meta)</em><br/>\n<strong>2°</strong> Lo Stato Sociale – <strong>Una vita in vacanza</strong><br/>\n<i>(Lodo Guenzi, Francesco Draicchio, Matteo Romagnoli, Alberto Guidetti, Enrico Roberto)</i><br/>\n<strong>3°</strong> Annalisa – <strong>Il mondo prima di te</strong><br/>\n<i>(Davide Simonetta, Alessandro Raina, Annalisa Scarrone)</i><br/>\n<strong>4°</strong> Ron – <strong>Almeno pensami</strong><br/>\n<em>(Lucio Dalla)</em><br/>\n<strong>5°</strong> Ornella Vanoni con Bungaro e Pacifico – <strong>Imparare ad amarsi</strong><br/>\n<i>(Cesare Chiodo, Andrea Fresa, Pacifico e Bungaro)</i><br/>\n<strong>6°</strong> Max Gazzè – <strong>La leggenda di Cristalda e Pizzomunno</strong><br/>\n<em>(Francesco De Benedittis, Francesco e\xa0Max Gazzè)</em><br/>\n<strong>7°</strong> Luca Barbarossa – <strong>Passame er sale</strong><br/>\n<em>(Luca Barbaros

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

20


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
lista_titoli_campioni[18] = 'Rivederti'
lista_titoli_campioni.append('Arrivedorci')
print(len(lista_titoli_campioni))

20


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Non mi avete fatto niente,"Andrea Febo, Fabrizio Moro, Ermal Meta"
1,Una vita in vacanza,"Lodo Guenzi, Francesco Draicchio, Matteo Romag..."
2,Il mondo prima di te,"Davide Simonetta, Alessandro Raina, Annalisa S..."
3,Almeno pensami,Lucio Dalla
4,Imparare ad amarsi,"Cesare Chiodo, Andrea Fresa, Pacifico e Bungaro"


In [ ]:
print(len(df_campioni))
print(len(data_2018))

20
20


In [ ]:
df_campioni['titolo'] = df_campioni['titolo'].str.lower()
data_2018['titolo'] = data_2018['titolo'].str.lower()

df_campioni['titolo'] = df_campioni['titolo'].str.replace('\’', '\'')
df_campioni['titolo'] = df_campioni['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2018 = pd.merge(data_2018, df_campioni, on = ['titolo'], how='left')
print(temp_2018.shape)
temp_2018

(20, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Ermal Meta e Fabrizio Moro,non mi avete fatto niente,"E. Meta, F. Moro e A. Febo",2018,"Andrea Febo, Fabrizio Moro, Ermal Meta"
1,2º,Lo Stato Sociale,una vita in vacanza,"A. Cazzola, F. Draicchio, L. Guenzi, A. Guidet...",2018,"Lodo Guenzi, Francesco Draicchio, Matteo Romag..."
2,3º,Annalisa,il mondo prima di te,"Annalisa, D. Simonetta e A. Raina",2018,"Davide Simonetta, Alessandro Raina, Annalisa S..."
3,4º,Ron,almeno pensami,L. Dalla,2018,Lucio Dalla
4,5º,Ornella Vanoni con Bungaro e Pacifico,imparare ad amarsi,"Bungaro, Pacifico, C. Chiodo e A. Fresa",2018,"Cesare Chiodo, Andrea Fresa, Pacifico e Bungaro"
5,6°,Max Gazzè,la leggenda di cristalda e pizzomunno,"F. Gazzè, M. Gazzè e F. De Benedittis",2018,"Francesco De Benedittis, Francesco e Max Gazzè"
6,7º,Luca Barbarossa,passame er sale,L. Barbarossa,2018,Luca Barbarossa
7,8º,Diodato e Roy Paci,adesso,Diodato,2018,Antonio Diodato
8,9º,The Kolors,"frida (mai, mai, mai)","D. Petrella, Dardust, A. Raina e Stash",2018,"Stash, Dario Faini, Alessandro Raina, Davide P..."
9,10º,Giovanni Caccamo,eterno,G. Caccamo e Cheope,2018,"Cheope, Giovanni Caccamo"


In [ ]:
temp_2018 = temp_2018.drop('autori', 1)
temp_2018.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Ermal Meta e Fabrizio Moro,non mi avete fatto niente,2018,"Andrea Febo, Fabrizio Moro, Ermal Meta"
1,2º,Lo Stato Sociale,una vita in vacanza,2018,"Lodo Guenzi, Francesco Draicchio, Matteo Romag..."
2,3º,Annalisa,il mondo prima di te,2018,"Davide Simonetta, Alessandro Raina, Annalisa S..."
3,4º,Ron,almeno pensami,2018,Lucio Dalla
4,5º,Ornella Vanoni con Bungaro e Pacifico,imparare ad amarsi,2018,"Cesare Chiodo, Andrea Fresa, Pacifico e Bungaro"


## 2019

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2019/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[6])
classifica_campioni

'<p><strong>1°</strong> Mahmood – <strong>Soldi</strong><br/>\n<em>(Dardust, Charlie Charles, Mahmood)</em><br/>\n<strong>2°</strong> Ultimo – <strong>I tuoi particolari</strong><br/>\n<i>(Ultimo)</i><br/>\n<strong>3°</strong> Il Volo – <strong>Musica che resta</strong><br/>\n<em>(Gianna Nannini, Piero Romitelli, Antonello Carozza, Emilio Munda, Pasquale Mammaro)<br/>\n</em><strong>4°</strong> Loredana Bertè – <strong>Cosa ti aspetti da me</strong><br/>\n<em>(Gaetano Curreri, Gerardo Pulli, Piero Romitelli)</em><br/>\n<strong>5°</strong> Simone Cristicchi – <strong>Abbi cura di me</strong><br/>\n<i>(Nicola Brunialti, Gabriele Ortenzi, Simone Cristicchi)</i><br/>\n<strong>6°</strong> Daniele Silvestri – <strong>Argentovivo</strong><br/>\n<em>(Manuel Agnelli, Fabio Rondanini, Rancore,\xa0Daniele Silvestri)</em><br/>\n<strong>7°</strong> Irama – <strong>La ragazza con il cuore di latta</strong><br/>\n<em>(Giulio Nenna, Andrea DB Bernardi, Giuseppe Colonnelli, Irama)<br/>\n</em><strong>8°<

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

24


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
lista_titoli_campioni[18] = 'Mi farò trovare pronto'
lista_titoli_campioni[19] = 'I ragazzi stanno bene'
lista_titoli_campioni[20] = 'Un po’ come la vita'
lista_titoli_campioni[21] = 'Le nostre anime di notte'
lista_titoli_campioni[22] = 'Parole nuove'
lista_titoli_campioni.append('Un’altra Luce')
print(len(lista_titoli_campioni))

24


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Soldi,"Dardust, Charlie Charles, Mahmood"
1,I tuoi particolari,Ultimo
2,Musica che resta,"Gianna Nannini, Piero Romitelli, Antonello Car..."
3,Cosa ti aspetti da me,"Gaetano Curreri, Gerardo Pulli, Piero Romitelli"
4,Abbi cura di me,"Nicola Brunialti, Gabriele Ortenzi, Simone Cri..."


In [ ]:
print(len(df_campioni))
print(len(data_2019))

24
24


In [ ]:
df_campioni['titolo'] = df_campioni['titolo'].str.lower()
data_2019['titolo'] = data_2019['titolo'].str.lower()

df_campioni['titolo'] = df_campioni['titolo'].str.replace('\’', '\'')
df_campioni['titolo'] = df_campioni['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2019 = pd.merge(data_2019, df_campioni, on = ['titolo'], how='left')
print(temp_2019.shape)
temp_2019

(24, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Mahmood,soldi,"Mahmood, Dardust e C. Charles",2019,"Dardust, Charlie Charles, Mahmood"
1,2º,Ultimo,i tuoi particolari,N. Moriconi,2019,Ultimo
2,3º,Il Volo,musica che resta,"G. Nannini, E. Munda, P. Romitelli, P. Mammaro...",2019,"Gianna Nannini, Piero Romitelli, Antonello Car..."
3,4º,Loredana Bertè,cosa ti aspetti da me,"G. Curreri, G. Pulli e P. Romitelli",2019,"Gaetano Curreri, Gerardo Pulli, Piero Romitelli"
4,5º,Simone Cristicchi,abbi cura di me,"S. Cristicchi, N. Brunialti e G. Ortenzi",2019,"Nicola Brunialti, Gabriele Ortenzi, Simone Cri..."
5,6º,Daniele Silvestri,argentovivo,"D. Silvestri, T. Iurcich, F. Rondanini e M. Ag...",2019,"Manuel Agnelli, Fabio Rondanini, Rancore, Dani..."
6,7º,Irama,la ragazza con il cuore di latta,"Irama, G. Colonnelli, A. Debernardi e G. Nenna",2019,"Giulio Nenna, Andrea DB Bernardi, Giuseppe Col..."
7,8º,Arisa,mi sento bene,"M. Buzzanca, L. Vizzini, R. Pippa e A. Flora",2019,"Matteo Buzzanca, Lorenzo Vizzini, Alessandra F..."
8,9º,Achille Lauro,rolls royce,"A. Lauro, D. Petrella, D. Dezi, D. Mungai e Bo...",2019,"Boss Doms, Davide Petrella, Daniele Mungai, Da..."
9,10º,Enrico Nigiotti,nonno hollywood,E. Nigiotti,2019,Enrico Nigiotti


In [ ]:
temp_2019 = temp_2019.drop('autori', 1)
temp_2019.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Mahmood,soldi,2019,"Dardust, Charlie Charles, Mahmood"
1,2º,Ultimo,i tuoi particolari,2019,Ultimo
2,3º,Il Volo,musica che resta,2019,"Gianna Nannini, Piero Romitelli, Antonello Car..."
3,4º,Loredana Bertè,cosa ti aspetti da me,2019,"Gaetano Curreri, Gerardo Pulli, Piero Romitelli"
4,5º,Simone Cristicchi,abbi cura di me,2019,"Nicola Brunialti, Gabriele Ortenzi, Simone Cri..."


## 2020

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2020/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[6])
classifica_campioni

'<p><strong>1°</strong> Diodato – <strong>Fai rumore</strong><br/>\n<i>(Edwyn Roberts, Antonio Diodato)</i><br/>\n<strong>2°</strong> Francesco Gabbani – <strong>Viceversa</strong><br/>\n<i>(Pacifico, Francesco Gabbani)</i><br/>\n<strong>3°</strong> Pinguini Tattici Nucleari – <strong>Ringo Starr</strong><br/>\n<em>(Riccardo Zanotti)<br/>\n</em><strong>4°</strong> Le Vibrazioni – <strong>Dov’è</strong><br/>\n<em>(Francesco Sarcina, Roberto Casalino, Davide Simonetta)</em><br/>\n<strong>5°</strong> Piero Pelù – <strong>Gigante</strong><br/>\n<i>(Luca Chiaravalli, Piero Pelù)</i><br/>\n<strong>6°</strong> Tosca – <strong>Ho amato tutto</strong><br/>\n<em>(Pietro Cantarelli)</em><br/>\n<strong>7°</strong> Elodie – <strong>Andromeda</strong><br/>\n<em>(Mahmood, Dardust)<br/>\n</em><strong>8°</strong> Achille Lauro – <strong>Me ne frego</strong><br/>\n<i>(Boss Doms, Davide Petrella, Daniele\xa0Dezi, Matteo Ciceroni, Daniele Mungai, Achille Lauro)</i><br/>\n<strong>9°</strong> Irene Grandi –

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

23


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
lista_titoli_campioni[18] = 'Baciami adesso'
lista_titoli_campioni[19] = 'Come mia madre'
lista_titoli_campioni[20] = 'Musica (e il resto scompare)'
lista_titoli_campioni[21] = 'No grazie'
lista_titoli_campioni.append('Lo sappiamo entrambi')
print(len(lista_titoli_campioni))

23


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Fai rumore,"Edwyn Roberts, Antonio Diodato"
1,Viceversa,"Pacifico, Francesco Gabbani"
2,Ringo Starr,Riccardo Zanotti
3,Dov’è,"Francesco Sarcina, Roberto Casalino, Davide Si..."
4,Gigante,"Luca Chiaravalli, Piero Pelù"


In [ ]:
classifica_non_finalisti = str(soup.find_all('p')[7])
classifica_non_finalisti

'<p><em><strong>Esclusi<br/>\n</strong>Bugo e Morgan – <strong>Sincero</strong><br/>\n(Andrea Bonomo, Simone Berolotti, Bugo, Morgan)<br/>\n</em></p>'

In [ ]:
lista_autori_non_finalisti = re.findall(r'\((.*?)\)', classifica_non_finalisti)
lista_autori_non_finalisti = list(filter(r.match, lista_autori_non_finalisti))
print(len(lista_autori_non_finalisti))

1


In [ ]:
lista_titoli_non_finalisti = re.findall(r'<strong>(.*?)<\/strong>', classifica_non_finalisti)
len(lista_titoli_non_finalisti)

1

In [ ]:
df_non_finalisti = pd.DataFrame()
df_non_finalisti['titolo']  = lista_titoli_non_finalisti
df_non_finalisti['Autori'] = lista_autori_non_finalisti
df_non_finalisti.head()

,titolo,Autori
0,Sincero,"Andrea Bonomo, Simone Berolotti, Bugo, Morgan"


In [ ]:
df = pd.concat([df_campioni, df_non_finalisti], ignore_index=True)

In [ ]:
print(len(df))
print(len(data_2020))

24
24


In [ ]:
df['titolo'] = df['titolo'].str.lower()
data_2020['titolo'] = data_2020['titolo'].str.lower()

df['titolo'] = df['titolo'].str.replace('\’', '\'')
df['titolo'] = df['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2020 = pd.merge(data_2020, df, on = ['titolo'], how='left')
print(temp_2019.shape)
temp_2020

(24, 5)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Diodato,fai rumore,Diodato e E. Roberts,2020,"Edwyn Roberts, Antonio Diodato"
1,2º,Francesco Gabbani,viceversa,F. Gabbani e Pacifico,2020,"Pacifico, Francesco Gabbani"
2,3º,Pinguini Tattici Nucleari,ringo starr,R. Zanotti,2020,Riccardo Zanotti
3,4º,Le Vibrazioni,dov'è,"R. Casalino, D. Simonetta e F. Sarcina",2020,"Francesco Sarcina, Roberto Casalino, Davide Si..."
4,5º,Piero Pelù,gigante,P. Pelù e L. Chiaravalli,2020,"Luca Chiaravalli, Piero Pelù"
5,6º,Tosca,ho amato tutto,P. Cantarelli,2020,Pietro Cantarelli
6,7º,Elodie,andromeda,Mahmood e Dardust,2020,"Mahmood, Dardust"
7,8º,Achille Lauro,me ne frego,"A. Lauro, D. Petrella, D. Dezi, D. Mungai, M. ...",2020,"Boss Doms, Davide Petrella, Daniele Dezi, Matt..."
8,9º,Irene Grandi,finalmente io,"V. Rossi, R. Casini, A. Righi e G. Curreri",2020,"Vasco Rossi, Gaetano Curreri, Roberto Casini, ..."
9,10º,Rancore,eden,Rancore e Dardust,2020,"Dardust, Rancore"


In [ ]:
temp_2020 = temp_2020.drop('autori', 1)
temp_2020.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Diodato,fai rumore,2020,"Edwyn Roberts, Antonio Diodato"
1,2º,Francesco Gabbani,viceversa,2020,"Pacifico, Francesco Gabbani"
2,3º,Pinguini Tattici Nucleari,ringo starr,2020,Riccardo Zanotti
3,4º,Le Vibrazioni,dov'è,2020,"Francesco Sarcina, Roberto Casalino, Davide Si..."
4,5º,Piero Pelù,gigante,2020,"Luca Chiaravalli, Piero Pelù"


## 2021

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2021/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[13])
classifica_campioni

'<p><strong>1°</strong> Maneskin – <strong>Zitti e buoni</strong><br/>\n<i>(Damiano David, Victoria De Angelis, Thomas Raggi, Ethan Torchio)</i><br/>\n<strong>2° </strong>Francesca Michielin e Fedez\xa0– <strong>Chiamami per nome</strong><br/>\n<i>(Francesca Michielin, Fedez, Dargen D’Amico, Mahmood, Alessandro Raina, Davide Simonetta)</i><br/>\n<strong>3°</strong> Ermal Meta – <strong>Un milione di cose da dirti</strong><br/>\n<em>(Ermal Meta, Roberto Cardelli)<br/>\n</em><strong>4°</strong> Colapesce e Dimartino – <strong>Musica leggerissima</strong><br/>\n<em>(Colapesce, Dimartino)</em><br/>\n<strong>5°</strong> Irama – <strong>La genesi del tuo\xa0</strong><b>colore</b><br/>\n<i>(Irama, Dardust, Giulio\xa0Nenna)</i><br/>\n<strong>6°</strong> Willie Peyote – <strong>Mai dire mai (la locura)</strong><br/>\n<em>(Willie Peyote, Carlo Cavalieri D’Oro, Daniel Bestonzo, Giuseppe Petrelli)</em><br/>\n<strong>7°</strong> Annalisa – <strong>Dieci</strong><br/>\n<em>(Annalisa, Davide Simonett

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

26


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]
lista_titoli_campioni[18] = 'Cuore amaro'
lista_titoli_campioni.append('Fiamme negli occhi')
lista_titoli_campioni.append('Momento perfetto')
lista_titoli_campioni.append('Quando trovo te')
lista_titoli_campioni.append('Arnica')
lista_titoli_campioni.append('E invece sì')
lista_titoli_campioni.append('Ora')
lista_titoli_campioni.append('Torno a te')
print(len(lista_titoli_campioni))

26


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Zitti e buoni,"Damiano David, Victoria De Angelis, Thomas Rag..."
1,Chiamami per nome,"Francesca Michielin, Fedez, Dargen D’Amico, Ma..."
2,Un milione di cose da dirti,"Ermal Meta, Roberto Cardelli"
3,Musica leggerissima,"Colapesce, Dimartino"
4,La genesi del tuo,"Irama, Dardust, Giulio Nenna"


In [ ]:
print(len(df_campioni))
print(len(data_2021))

26
26


In [ ]:
df_campioni['titolo'] = df_campioni['titolo'].str.lower()
data_2021['titolo'] = data_2021['titolo'].str.lower()

df_campioni['titolo'] = df_campioni['titolo'].str.replace('\’', '\'')
df_campioni['titolo'] = df_campioni['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2021 = pd.merge(data_2021, df_campioni, on = ['titolo'], how='left')
print(temp_2021.shape)
temp_2021

(26, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Måneskin,zitti e buoni,"D. David, T. Raggi, V. De Angelis e E. Torchio",2021,"Damiano David, Victoria De Angelis, Thomas Rag..."
1,2º,Francesca Michielin e Fedez,chiamami per nome,"F. Michielin, Fedez, D. D’Amico, A. Mahmoud, A...",2021,"Francesca Michielin, Fedez, Dargen D’Amico, Ma..."
2,3º,Ermal Meta,un milione di cose da dirti,E. Meta e R. Cardelli,2021,"Ermal Meta, Roberto Cardelli"
3,4º,Colapesce Dimartino,musica leggerissima,Colapesce e Dimartino,2021,"Colapesce, Dimartino"
4,5º,Irama[20],la genesi del tuo colore,"Irama, Dardust e G. Nenna",2021,NaN
5,6º,Willie Peyote,mai dire mai (la locura),"W. Peyote, C. Cavalieri D’Oro, D. Bestonzo e G...",2021,"Willie Peyote, Carlo Cavalieri D’Oro, Daniel B..."
6,7º,Annalisa,dieci,"Annalisa, D. Simonetta, P. Antonacci e D. D’Amico",2021,"Annalisa, Davide Simonetta, Paolo Antonacci, D..."
7,8º,Madame,voce,"Madame, Dardust e E. Botta",2021,"Madame, Dardust, Enrico Botta"
8,9º,Orietta Berti,quando ti sei innamorato,"F. Boccia, C. Esposito e M. Rettani",2021,"Francesco Boccia, Ciro Esposito, Marco Rettani"
9,10º,Arisa,potevi fare di più,G. D'Alessio,2021,Gigi D’Alessio


In [ ]:
temp_2021 = temp_2021.drop('autori', 1)
temp_2021.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Måneskin,zitti e buoni,2021,"Damiano David, Victoria De Angelis, Thomas Rag..."
1,2º,Francesca Michielin e Fedez,chiamami per nome,2021,"Francesca Michielin, Fedez, Dargen D’Amico, Ma..."
2,3º,Ermal Meta,un milione di cose da dirti,2021,"Ermal Meta, Roberto Cardelli"
3,4º,Colapesce Dimartino,musica leggerissima,2021,"Colapesce, Dimartino"
4,5º,Irama[20],la genesi del tuo colore,2021,NaN


## 2022

In [ ]:
query = 'https://recensiamomusica.com/sanremo-per-tutti-la-storia-del-festival-2022/'
html = requests.get(query).text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classifica_campioni = str(soup.find_all('p')[14])
classifica_campioni

'<p><strong>1°</strong> Mahmood e Blanco – <strong>Brividi</strong><br/>\n<i>(Mahmood, Blanco, Michelangelo)</i><br/>\n<strong>2° </strong>Elisa – <strong>O forse sei tu</strong><br/>\n<i>(Elisa, Davide Petrella)</i><br/>\n<strong>3°</strong> Gianni Morandi – <strong>Apri tutte le porte</strong><br/>\n<em>(Jovanotti, Riccardo Onori)<br/>\n</em><strong>4°</strong> Irama – <strong>Ovunque sarai</strong><br/>\n<em>(Irama, Giulio Nenna, Shablo, Vincenzo Luca Faraone, Giuseppe Colonnelli)</em><br/>\n<strong>5°</strong> Sangiovanni – <b>Farfalle</b><br/>\n<i>(Sangiovanni, Alessandro La Cava, Stefano Tognini)</i><br/>\n<strong>6°</strong> Emma – <strong>Ogni\xa0volta è così</strong><br/>\n<em>(Davide Petrella, Emma, Dario Faini)</em><br/>\n<strong>7°</strong> La Rappresentante di Lista – <strong>Ciao ciao</strong><br/>\n<em>(Veronica Lucchesi, Dario Mangiaracina, Roberto Calabrese, Roberto Cammarata, Carlmelo Drago, Simone Privitera)<br/>\n</em><strong>8°</strong> Massimo Ranieri – <strong>Le

In [ ]:
lista_autori_campioni = re.findall(r'\((.*?)\)', classifica_campioni)

r = re.compile("(?=.*[A-Z])")
lista_autori_campioni = list(filter(r.match, lista_autori_campioni))
print(len(lista_autori_campioni))

25


In [ ]:
x = re.findall(r'<strong>(.*?)<\/strong>', classifica_campioni)

lista_titoli_campioni = x[1::2]

j = 0
mancanti = ['Farfalle', 'Ogni volta è così', 'Ciao ciao', 'Lettera di là dal mare', 'Dove si balla', 
            'Inverno dei fiori', 'Virale', 'Sei tu', 'Perfetta così', 'Domenica', 'Ti amo non lo so dire', 
            'Chimica', 'Insuperabile', 'Voglio amarti', 'Tuo padre, mia madre, Lucia']
for i in range (4, len(lista_titoli_campioni)):
  lista_titoli_campioni[i] = mancanti[j]
  j +=1

lista_titoli_campioni.append('Abbi cura di te')
lista_titoli_campioni.append('Ora e qui')
lista_titoli_campioni.append('Tantissimo')
lista_titoli_campioni.append('Miele')
lista_titoli_campioni.append('Duecentomila ore')
lista_titoli_campioni.append('Sesso occasionale')

print(len(lista_titoli_campioni))

25


In [ ]:
df_campioni = pd.DataFrame()
df_campioni['titolo']  = lista_titoli_campioni
df_campioni['Autori'] = lista_autori_campioni
df_campioni.head()

,titolo,Autori
0,Brividi,"Mahmood, Blanco, Michelangelo"
1,O forse sei tu,"Elisa, Davide Petrella"
2,Apri tutte le porte,"Jovanotti, Riccardo Onori"
3,Ovunque sarai,"Irama, Giulio Nenna, Shablo, Vincenzo Luca Far..."
4,Farfalle,"Sangiovanni, Alessandro La Cava, Stefano Tognini"


In [ ]:
print(len(df_campioni))
print(len(data_2022))

25
25


In [ ]:
df_campioni['titolo'] = df_campioni['titolo'].str.lower()
data_2022['titolo'] = data_2022['titolo'].str.lower()

df_campioni['titolo'] = df_campioni['titolo'].str.replace('\’', '\'')
df_campioni['titolo'] = df_campioni['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
temp_2022 = pd.merge(data_2022, df_campioni, on = ['titolo'], how='left')
print(temp_2022.shape)
temp_2022

(25, 6)


,Posizione,Interprete,titolo,autori,edizione,Autori
0,1º,Mahmood e Blanco,brividi,"Mahmood, Blanco e Michelangelo",2022,"Mahmood, Blanco, Michelangelo"
1,2º,Elisa,o forse sei tu,Elisa e D. Petrella,2022,"Elisa, Davide Petrella"
2,3º,Gianni Morandi,apri tutte le porte,L. Jovanotti e R. Onori,2022,"Jovanotti, Riccardo Onori"
3,4º,Irama,ovunque sarai,"Irama, G. Nenna, Shablo, V. L. Faraone e G. Co...",2022,"Irama, Giulio Nenna, Shablo, Vincenzo Luca Far..."
4,5º,Sangiovanni,farfalle,"Sangiovanni, A. La Cava e S. Tognini",2022,"Sangiovanni, Alessandro La Cava, Stefano Tognini"
5,6º,Emma,ogni volta è così,"D. Petrella, Emma e D. Faini",2022,"Davide Petrella, Emma, Dario Faini"
6,7º,La Rappresentante di Lista,ciao ciao,"V. Lucchesi, D. Mangiaracina, R. Calabrese, R....",2022,"Veronica Lucchesi, Dario Mangiaracina, Roberto..."
7,8º,Massimo Ranieri,lettera di là dal mare,F. Ilacqua,2022,Fabio Ilaqua
8,9º,Dargen D'Amico,dove si balla,"D. D'Amico, E. Roberts, G. Fazio e A. Bonomo",2022,"Dargen D’Amico, Edwyn Roberts, Gianluigi Fazio..."
9,10º,Michele Bravi,inverno dei fiori,"A. A. Vella, M. Bravi, Cheope, F. Catitti e F....",2022,"Raige, Michele Bravi, Federica Abbate, Cheope,..."


In [ ]:
temp_2022 = temp_2022.drop('autori', 1)
temp_2022.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Mahmood e Blanco,brividi,2022,"Mahmood, Blanco, Michelangelo"
1,2º,Elisa,o forse sei tu,2022,"Elisa, Davide Petrella"
2,3º,Gianni Morandi,apri tutte le porte,2022,"Jovanotti, Riccardo Onori"
3,4º,Irama,ovunque sarai,2022,"Irama, Giulio Nenna, Shablo, Vincenzo Luca Far..."
4,5º,Sangiovanni,farfalle,2022,"Sangiovanni, Alessandro La Cava, Stefano Tognini"


# Merge con Spotify

In [ ]:
# vengono uniti nuovamente in un formato corretto
final = pd.concat([temp_2013, temp_2014, temp_2015, temp_2016, temp_2017, temp_2018, temp_2019, temp_2020, temp_2021, temp_2022], ignore_index=True)
print(final.shape)
final.head()

(237, 5)


,Posizione,Interprete,titolo,edizione,Autori
0,1º,Marco Mengoni,l'essenziale,2013,"Roberto Casalino, Francesco De Benedittis, Mar..."
1,2º,Elio e le Storie Tese,la canzone mononota,2013,"Sergio Conforti, Davide Civaschi, Nicola Fasan..."
2,3º,Modà,se si potesse non morire,2013,Kekko Silvestre
3,4º,Malika Ayane,e se poi,2013,Giuliano Sangiorgi
4,5º,Raphael Gualazzi,sai (ci basta un sogno),2013,Raphael Gualazzi


In [ ]:
# lettura file tracce proveniente da spotify
tracce_final = pd.read_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/tracce_sanremo_V2.csv', encoding = 'ISO-8859-1')
print(tracce_final.shape)
tracce_final.head()

(229, 14)


,esplicita,id_track,titolo,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo,edizione
0,False,5klQSk2bLb0U3vnvGJUZf4,L'essenziale,0.556,0.632,6.0,-4.438,maggiore,0.0359,0.152,126.0,3.65,4.0,2013
1,False,4EHbsQ6mzg1eReuAFh0b51,La canzone Mononota,0.481,0.597,0.0,-8.126,maggiore,0.3610,0.699,160.0,4.98,4.0,2013
2,False,4zjBp0Jp1Ij3e1flAn2BPq,Se Si Potesse Non Morire,0.467,0.688,11.0,-6.336,maggiore,0.0367,0.197,130.0,4.31,4.0,2013
3,False,0qgOTifFiRDywPib2Rzy42,E Se Poi,0.489,0.677,11.0,-5.829,minore,0.0293,0.482,110.0,4.73,4.0,2013
4,False,4xz34M5LL2nt3RrZVUc0Pz,Sai (Ci basta un sogno),0.322,0.332,1.0,-9.233,maggiore,0.0371,0.248,78.0,4.06,4.0,2013


In [ ]:
# INIZIO PRE-PROCESSING PER POTER FARE MERGE
# tutti i titoli in lower case
tracce_final['titolo'] = tracce_final['titolo'].str.lower()

tracce_final['titolo'] = tracce_final['titolo'].str.replace('\’', '\'')
tracce_final['titolo'] = tracce_final['titolo'].str.replace('e\'', 'è')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


Primo test merge


In [ ]:
merged_data = pd.merge(final, tracce_final, on = ['titolo', 'edizione'], how='left')
print(merged_data.shape)
merged_data.head()

(237, 17)


,Posizione,Interprete,titolo,edizione,Autori,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
0,1º,Marco Mengoni,l'essenziale,2013,"Roberto Casalino, Francesco De Benedittis, Mar...",False,5klQSk2bLb0U3vnvGJUZf4,0.556,0.632,6.0,-4.438,maggiore,0.0359,0.152,126.0,3.65,4.0
1,2º,Elio e le Storie Tese,la canzone mononota,2013,"Sergio Conforti, Davide Civaschi, Nicola Fasan...",False,4EHbsQ6mzg1eReuAFh0b51,0.481,0.597,0.0,-8.126,maggiore,0.3610,0.699,160.0,4.98,4.0
2,3º,Modà,se si potesse non morire,2013,Kekko Silvestre,False,4zjBp0Jp1Ij3e1flAn2BPq,0.467,0.688,11.0,-6.336,maggiore,0.0367,0.197,130.0,4.31,4.0
3,4º,Malika Ayane,e se poi,2013,Giuliano Sangiorgi,False,0qgOTifFiRDywPib2Rzy42,0.489,0.677,11.0,-5.829,minore,0.0293,0.482,110.0,4.73,4.0
4,5º,Raphael Gualazzi,sai (ci basta un sogno),2013,Raphael Gualazzi,False,4xz34M5LL2nt3RrZVUc0Pz,0.322,0.332,1.0,-9.233,maggiore,0.0371,0.248,78.0,4.06,4.0


In [ ]:
merged_data['chiave'].isna().sum()

33

In [ ]:
merged_data[merged_data['chiave'].isna()]

,Posizione,Interprete,titolo,edizione,Autori,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
9,10º,Maria Nazionale,è colpa mia,2013,"Peppe Servillo, Fausto Mesolella",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13º,Simona Molinari con Peter Cincotti,la felicità,2013,"Carlo Avarello, Maurizio Vultaggio, Simona Mol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NF,Simona Molinari con Peter Cincotti,dr. jekyll mr. hyde,2013,"Lelio Luttazzi, Alberto Zeppieri",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NF,Daniele Silvestri,il bisogno di te (ricatto d'onor),2013,Daniele Silvestri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NF,Maria Nazionale,quando non parlo,2013,Enzo Gragnaniello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,6º,Perturbazione,l'unica,2014,"Tommaso Cerasuolo, Gigi Giancursi, Rossano e C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,8º,Frankie hi-nrg mc,pedala,2014,"Leonardo Beccafichi, Carolina Galbignani, Fran...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,12º,Antonella Ruggiero,da lontano,2014,"Alessandro Graziano, Roberto Colombo, Antonio ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,13º,Ron,sing in the rain,2014,"Mattia Del Forno, Ron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NF,Perturbazione,l'italia vista dal bar,2014,"Tommaso Cerasuolo, Gigi Giancursi, Rossano e C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Facendo una prima prova si nota che i non match sono per titoli che contengono indicazioni sull'edizione di sanremo, sui featuring e altro.

**Prima correzione: edizione del festival**

In [ ]:
df = tracce_final.copy()

In [ ]:
# si cercano titoli che contengono espressioni come 
#  - festival di sanremo 2014,  - sanremo 2019 ecc e si sostituiscono con il nulla
for i in range(0, len(df)):
  if df["titolo"].str.contains(r'( - ).*(sanremo)')[i] == True:
    df["titolo"][i] = re.sub('( - ).*(sanremo) \d*', '', df["titolo"][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
merged_data = pd.merge(final, df, on = ['titolo', 'edizione'], how='left')
print(merged_data.shape)

(237, 17)


In [ ]:
print(merged_data['chiave'].isna().sum())
merged_data[merged_data['chiave'].isna()]

24


,Posizione,Interprete,titolo,edizione,Autori,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
9,10º,Maria Nazionale,è colpa mia,2013,"Peppe Servillo, Fausto Mesolella",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13º,Simona Molinari con Peter Cincotti,la felicità,2013,"Carlo Avarello, Maurizio Vultaggio, Simona Mol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NF,Simona Molinari con Peter Cincotti,dr. jekyll mr. hyde,2013,"Lelio Luttazzi, Alberto Zeppieri",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NF,Daniele Silvestri,il bisogno di te (ricatto d'onor),2013,Daniele Silvestri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NF,Maria Nazionale,quando non parlo,2013,Enzo Gragnaniello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,12º,Antonella Ruggiero,da lontano,2014,"Alessandro Graziano, Roberto Colombo, Antonio ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,13º,Ron,sing in the rain,2014,"Mattia Del Forno, Ron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NF,Antonella Ruggiero,quando balliamo,2014,"Simone Lenzi, Roberto Colombo, Antonella Ruggiero",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NF,Ron,un abbraccio unico,2014,Ron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,6º,Marco Masini,che giorno è,2015,"Federica Camba, Daniele Coro, Marco Masini",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Si sono risolti una decina di non match

**Seconda correzione: featuring**

In [ ]:
df2 = df.copy()

In [ ]:
for i in range(0, len(df2)):
  if df2["titolo"].str.contains(r'\((feat.*)\)')[i] == True or df2["titolo"].str.contains(r'\((with.*)\)')[i] == True:
    df2["titolo"][i] = re.sub('\s\((feat.*)\)', '', df2["titolo"][i])
    df2["titolo"][i] = re.sub('\s\((with.*)\)', '', df2["titolo"][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
merged_data = pd.merge(final, df2, on = ['titolo', 'edizione'], how='left')
print(merged_data.shape)

(237, 17)


In [ ]:
print(merged_data['chiave'].isna().sum())
merged_data[merged_data['chiave'].isna()]

15


,Posizione,Interprete,titolo,edizione,Autori,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
9,10º,Maria Nazionale,è colpa mia,2013,"Peppe Servillo, Fausto Mesolella",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NF,Daniele Silvestri,il bisogno di te (ricatto d'onor),2013,Daniele Silvestri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NF,Maria Nazionale,quando non parlo,2013,Enzo Gragnaniello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,12º,Antonella Ruggiero,da lontano,2014,"Alessandro Graziano, Roberto Colombo, Antonio ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,13º,Ron,sing in the rain,2014,"Mattia Del Forno, Ron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NF,Antonella Ruggiero,quando balliamo,2014,"Simone Lenzi, Roberto Colombo, Antonella Ruggiero",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NF,Ron,un abbraccio unico,2014,Ron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NF,Biggio e Mandelli,vita d'inferno,2015,"Martino Ferro, Francesco Mandelli, Fabrizio Bi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,NF,Lara Fabian,voce,2015,"Fio Zanotti, Cristiano Cremonini, Lara Fabian",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,1º,Francesco Gabbani,occidentali's karma,2017,"Fabio Ilaqua, Luca Chiaravalli, Filippo e Fran...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Risolti altri problemi di non match

**terza correzione: edit e versioni**

In [ ]:
df3 = df2.copy()

In [ ]:
for i in range(0, len(df3)):
  if df3["titolo"].str.contains(r'( - ).*(edit)')[i] == True or df3["titolo"].str.contains(r'( - ).*(version)')[i] == True:
    df3["titolo"][i] = re.sub('( - ).*(edit)', '', df3["titolo"][i])
    df3["titolo"][i] = re.sub('( - ).*(version)', '', df3["titolo"][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
merged_data = pd.merge(final, df3, on = ['titolo', 'edizione'], how='left')
print(merged_data.shape)

(237, 17)


In [ ]:
print(merged_data['chiave'].isna().sum())
merged_data[merged_data['chiave'].isna()]

13


,Posizione,Interprete,titolo,edizione,Autori,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
9,10º,Maria Nazionale,è colpa mia,2013,"Peppe Servillo, Fausto Mesolella",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NF,Daniele Silvestri,il bisogno di te (ricatto d'onor),2013,Daniele Silvestri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NF,Maria Nazionale,quando non parlo,2013,Enzo Gragnaniello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,12º,Antonella Ruggiero,da lontano,2014,"Alessandro Graziano, Roberto Colombo, Antonio ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,13º,Ron,sing in the rain,2014,"Mattia Del Forno, Ron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NF,Antonella Ruggiero,quando balliamo,2014,"Simone Lenzi, Roberto Colombo, Antonella Ruggiero",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NF,Ron,un abbraccio unico,2014,Ron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NF,Biggio e Mandelli,vita d'inferno,2015,"Martino Ferro, Francesco Mandelli, Fabrizio Bi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,NF,Lara Fabian,voce,2015,"Fio Zanotti, Cristiano Cremonini, Lara Fabian",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,24º,Nino D'Angelo e Livio Cori,un'altra luce,2019,"Massimiliano Dagani, Giancaludio Facchiolla, F...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Correzioni finali: alcuni casi particolari**

In [ ]:
df4 = df3.copy()

In [ ]:
for i in range(0, len(df4)):
  if df4["titolo"].str.contains(r'\se\'\s')[i] == True or df4["titolo"].str.contains(r'\(.*#\d*\)')[i] == True or df4["titolo"].str.contains(r'version')[i] == True:
    df4["titolo"][i] = re.sub('\s\(.*#\d*\)', '', df4["titolo"][i])
    df4["titolo"][i] = re.sub('version', '', df4["titolo"][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
merged_data = pd.merge(final, df4, on = ['titolo', 'edizione'], how='left')
print(merged_data.shape)

(237, 17)


In [ ]:
print(merged_data['chiave'].isna().sum())
merged_data[merged_data['chiave'].isna()]

11


,Posizione,Interprete,titolo,edizione,Autori,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
9,10º,Maria Nazionale,è colpa mia,2013,"Peppe Servillo, Fausto Mesolella",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NF,Daniele Silvestri,il bisogno di te (ricatto d'onor),2013,Daniele Silvestri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NF,Maria Nazionale,quando non parlo,2013,Enzo Gragnaniello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,12º,Antonella Ruggiero,da lontano,2014,"Alessandro Graziano, Roberto Colombo, Antonio ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,13º,Ron,sing in the rain,2014,"Mattia Del Forno, Ron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NF,Antonella Ruggiero,quando balliamo,2014,"Simone Lenzi, Roberto Colombo, Antonella Ruggiero",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NF,Ron,un abbraccio unico,2014,Ron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NF,Biggio e Mandelli,vita d'inferno,2015,"Martino Ferro, Francesco Mandelli, Fabrizio Bi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,NF,Lara Fabian,voce,2015,"Fio Zanotti, Cristiano Cremonini, Lara Fabian",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,24º,Bugo,e invece sì,2021,"Bugo, Andrea Bonomo, Simone Bertolotti",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# si salva una copia contenente anche interpreti e autori, che però verrà utilizzata solo come controllo
merged_data.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/tracce_per_controllo.csv', index = False)

**Tabella finale**

In [ ]:
# si eliminano colonne interprete e autori perchè nel grafo finale verranno utilizzate le tabelle pivot,
# che associano correttamente i potenziali duetti
merged_data_final = merged_data.drop('Interprete', 1)
merged_data_final = merged_data_final.drop('Autori', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [ ]:
#tabella finale
merged_data_final.head()

,Posizione,titolo,edizione,esplicita,id_track,ballabilità,energia,chiave,rumorosità,tonalità,parlato,positività,tempo,durata,ritmo
0,1º,l'essenziale,2013,False,5klQSk2bLb0U3vnvGJUZf4,0.556,0.632,6.0,-4.438,maggiore,0.0359,0.152,126.0,3.65,4.0
1,2º,la canzone mononota,2013,False,4EHbsQ6mzg1eReuAFh0b51,0.481,0.597,0.0,-8.126,maggiore,0.3610,0.699,160.0,4.98,4.0
2,3º,se si potesse non morire,2013,False,4zjBp0Jp1Ij3e1flAn2BPq,0.467,0.688,11.0,-6.336,maggiore,0.0367,0.197,130.0,4.31,4.0
3,4º,e se poi,2013,False,0qgOTifFiRDywPib2Rzy42,0.489,0.677,11.0,-5.829,minore,0.0293,0.482,110.0,4.73,4.0
4,5º,sai (ci basta un sogno),2013,False,4xz34M5LL2nt3RrZVUc0Pz,0.322,0.332,1.0,-9.233,maggiore,0.0371,0.248,78.0,4.06,4.0


In [ ]:
merged_data_final.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/tracce_final.csv', index = False)

# Preparazione pivot per autori

In [ ]:
pivot = merged_data[['id_track', 'Autori']].copy()
pivot.head()

,id_track,Autori
0,5klQSk2bLb0U3vnvGJUZf4,"Roberto Casalino, Francesco De Benedittis, Mar..."
1,4EHbsQ6mzg1eReuAFh0b51,"Sergio Conforti, Davide Civaschi, Nicola Fasan..."
2,4zjBp0Jp1Ij3e1flAn2BPq,Kekko Silvestre
3,0qgOTifFiRDywPib2Rzy42,Giuliano Sangiorgi
4,4xz34M5LL2nt3RrZVUc0Pz,Raphael Gualazzi


In [ ]:
pivot.dropna(inplace=True)

In [ ]:
pivot['Autori'] = pivot['Autori'].apply(lambda x: x.replace('\xa0', ' '))
pivot['Autori'] = pivot['Autori'].apply(lambda x: x.replace('amp;', ''))

In [ ]:
pivot['Autori'] = pivot['Autori'].apply(lambda x: x.split(', '))
pivot = pivot.explode('Autori')[['id_track', 'Autori']]
pivot = pivot.reset_index(drop=True)

In [ ]:
pivot.head()

,id_track,Autori
0,5klQSk2bLb0U3vnvGJUZf4,Roberto Casalino
1,5klQSk2bLb0U3vnvGJUZf4,Francesco De Benedittis
2,5klQSk2bLb0U3vnvGJUZf4,Marco Mengoni
3,4EHbsQ6mzg1eReuAFh0b51,Sergio Conforti
4,4EHbsQ6mzg1eReuAFh0b51,Davide Civaschi


In [ ]:
for j in range (0, len(pivot['Autori'])):
    if ' e ' in pivot['Autori'][j]:
      if pivot['Autori'][j] == 'Pacifico e Bungaro':
        d = pivot['Autori'][j].split(' ')
        primo = d[0]
        secondo = d[2]
        new = primo + ", " + secondo
        pivot['Autori'][j] = new

      elif pivot['Autori'][j]!='Takagi e Ketra':
        d = pivot['Autori'][j].split(' ')
        primo = d[0] + " " + d[-1]
        secondo = d[2] + " " + d[-1]
        new = primo + ", " + secondo
        pivot['Autori'][j] = new
            
pivot['Autori'] = pivot['Autori'].apply(lambda x: x.split(', '))
pivot = pivot.explode('Autori')[['id_track', 'Autori']]
pivot = pivot.reset_index(drop=True)

In [ ]:
print(pivot.shape)
pivot.head()

(631, 2)


,id_track,Autori
0,5klQSk2bLb0U3vnvGJUZf4,roberto casalino
1,5klQSk2bLb0U3vnvGJUZf4,francesco de benedittis
2,5klQSk2bLb0U3vnvGJUZf4,marco mengoni
3,4EHbsQ6mzg1eReuAFh0b51,sergio conforti
4,4EHbsQ6mzg1eReuAFh0b51,davide civaschi


In [ ]:
pivot.Autori=pivot.Autori.apply(lambda x: str.lower(x))

In [ ]:
print(pivot[pivot['Autori']==""])
pivot.drop([522], axis=0, inplace=True)
pivot=pivot.reset_index(drop=True)

                   id_track Autori
522  6PjFhlsfcwcmngrd4MK23Q       


In [ ]:
pivot.isna().sum()

id_track    0
Autori      0
dtype: int64

In [ ]:
pivot.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/autori_quality.csv', index=False)

DA QUA IN GIù, SPOSTARE DOPO NELLA PARTE DI QUALITà PER RISOLVERE I PROBLEMI DI CONSISTENZA; RICORDARSI DI MODOFICARE LA RIGA CONTENENTE EDWIN ROBERTS IN AUTORI_QUALITY

In [ ]:
pivot[pivot['Autori']=="annalisa scarrone"]

,id_track,Autori
266,0qM6TGkkinhKFllbcRfL8u,annalisa scarrone


In [ ]:
pivot[pivot['Autori']=="dargen d’amico"]

,id_track,Autori
458,0ZqHo6eTBHUcOVjZzPvftT,dargen d’amico
473,2lBLzzEARN8mGgHFaXYKKk,dargen d’amico
552,3FVa0R2evbgjVOd3bejudr,dargen d’amico


In [ ]:
pivot[pivot['Autori']=="antonio diodato"]

,id_track,Autori
274,4Apr4hRIZhaXiitbVgS1kH,antonio diodato
393,5OaBhC8NjdqDhHF9I6r9lS,antonio diodato


In [ ]:
pivot[pivot['Autori']=="samuel umberto romano"]

,id_track,Autori
225,20cnvre3xAna8nTSRKKlZo,samuel umberto romano


In [ ]:
pivot[pivot['Autori']=="francesco motta"]

,id_track,Autori
353,7I4NSuAPhDmnOZbDet3Blt,francesco motta


In [ ]:
pivot[pivot['Autori']=="marco anastasio"]

,id_track,Autori
424,3mj23ZA8LARtZSSVJxIZOi,marco anastasio


In [ ]:
pivot[pivot['Autori']=="riki marcuzzo"]

,id_track,Autori
447,5DKbPEGPIhTk1LouggxExD,riki marcuzzo


In [ ]:
pivot[pivot['Autori']=="al bano carrisi"]

,id_track,Autori
244,2RSVqZNFZyTZimeJMSkPQP,al bano carrisi


In [ ]:
pivot[pivot['Autori']=="stefano belisari"]

,id_track,Autori
176,73wYIL1vixAtfVWOi5tTi4,stefano belisari
305,4UmLL12rZPLHKkQNW4mzoI,stefano belisari


In [ ]:
pivot[pivot['Autori']=="elio"]

,id_track,Autori
6,4EHbsQ6mzg1eReuAFh0b51,elio
38,3TtellNIzeM8AaU0k70UFm,elio


In [ ]:
pivot.loc[469, 'Autori'] = "dargen d'amico"
pivot.loc[484, 'Autori'] = "dargen d'amico"
pivot.loc[563, 'Autori'] = "dargen d'amico"

In [ ]:
pivot.loc[274, 'Autori'] = 'annalisa'

In [ ]:
pivot.loc[232, 'Autori'] = 'samuel'

In [ ]:
pivot.loc[434, 'Autori'] = 'anastasio'

In [ ]:
pivot.loc[458, 'Autori'] = 'riki'

In [ ]:
pivot.loc[363, 'Autori'] = 'motta'

In [ ]:
pivot.loc[284, 'Autori'] = 'diodato'
pivot.loc[403, 'Autori'] = 'diodato'

In [ ]:
pivot.loc[252, 'Autori'] = 'al bano'

In [ ]:
pivot.loc[6, 'Autori'] = 'stefano belisari'
pivot.loc[39, 'Autori'] = 'stefano belisari'

In [ ]:
len(pivot['Autori'])

631

In [ ]:
pivot['Autori'].tolist()

['roberto casalino',
 'francesco de benedittis',
 'marco mengoni',
 'sergio conforti',
 'davide civaschi',
 'nicola fasani',
 'stefano belisari',
 'kekko silvestre',
 'giuliano sangiorgi',
 'raphael gualazzi',
 'daniele silvestri',
 'francesco de benedittis',
 'max gazzè',
 'francesco gazzè',
 'francesco bianconi',
 'luca chiaravalli',
 'lisette gonzales-alea',
 'dario faini',
 'antonio galbiati',
 'leo pari',
 'simone cristicchi',
 'giovanni gulino',
 'mattia boschi',
 'carmelo pipitone',
 'paolo pischedda',
 'ivan paolini',
 'carlo avarello',
 'maurizio vultaggio',
 'simona molinari',
 'gennaro della volpe',
 'paolo polcari',
 'gennaro tesone',
 'gianna nannini',
 'pacifico',
 'davide tagliapietra',
 'kekko silvestre',
 'sergio conforti',
 'davide civaschi',
 'nicola fasani',
 'stefano belisari',
 'giovanni gulino',
 'mattia boschi',
 'carmelo pipitone',
 'paolo pischedda',
 'ivan paolini',
 'lelio luttazzi',
 'alberto zeppieri',
 'matteo buzzanca',
 'max gazzè',
 'francesco gazzè',


In [ ]:
pivot.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/pivot_autori_final.csv', index = False)

# Creazione tabella edizioni

In [ ]:
anni = []
for i in range (2013, 2023):
  anni.append(i)
anni

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [ ]:
df_edizioni = pd.DataFrame(anni, columns=['anno'])

In [ ]:
df_edizioni

,anno
0,2013
1,2014
2,2015
3,2016
4,2017
5,2018
6,2019
7,2020
8,2021
9,2022


In [ ]:
df_edizioni.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/edizione.csv', index = False)

# Vecchia modalità pivot NON RUNNARE

In [ ]:
# lista_totale = []
# for i in range (0, len(pivot)):
#   if type(pivot['Autori'][i])==str:
#     autori = pivot['Autori'].str.split(',')[i]
#     for j in range (0, len(autori)):
#       lista_totale.append(autori[j])

In [ ]:
# print(len(lista_totale))
# #lista_totale

In [ ]:
# pivot['times'] = 0
# for i in range(0, len(pivot)):
#   if type(pivot['Autori'][i])==str:
#     pivot['times'][i] = len(pivot['Autori'].str.split(',')[i])

In [ ]:
# pivot = pivot.loc[pivot.index.repeat(pivot.times)].reset_index()
# len(pivot)

In [ ]:
# pivot[100:150]

In [ ]:
# pivot['new_autori'] = lista_totale

# pivot

In [ ]:
# pivot_autori = pivot[['id.track', 'new_autori']].copy()

In [ ]:
# pivot_autori['new_autori'] = pivot_autori['new_autori'].str.strip()

In [ ]:
# pivot_autori.head()

In [ ]:
# print(pivot_autori['id.track'].isna().sum())
# pivot_autori[pivot_autori['id.track'].isna()]['new_autori'].tolist()

In [ ]:
# merged_data[merged_data['chiave'].isna()]['Autori'].tolist()

In [ ]:
# pivot_autori.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/pivot_autori.csv', index = False)

Queste sono le stesse canzoni che nella parte precedente risultano con l'attributo chiave nullo. In questo caso le righe sono 30 al posto che 13 perchè aveno espando la stringa degli autori uno per riga, e essendoci diversi autori per una canzone, il numero cresce. Confrontando con la tabella merged data si nota che gli autori sono gli stessi.